In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import re
import os
import random
import math
import csv 

from tqdm import tqdm
import matplotlib.pyplot as plt

print(tf.__version__)

2.6.0


# Step 2. 데이터 정제

In [2]:

# CSV 파일 경로
file_path = "ChatbotData.csv"  # 파일 경로를 실제 파일 경로로 변경하세요.

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 데이터프레임 출력
print(df.head())  # 상위 5개 행 출력
print("\n")

df = pd.read_csv(file_path, usecols=['Q', 'A'])

print(df.head())  # 상위 5개 행 출력


                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


                 Q            A
0           12시 땡!   하루가 또 가네요.
1      1지망 학교 떨어졌어    위로해 드립니다.
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.
4          PPL 심하네   눈살이 찌푸려지죠.


In [3]:
questions =[] 
answers =[]

questions = df['Q']
answers = df['A']

print(questions[:5])
print("\n")
print(answers[:5])

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object


0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object


In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣0-9,.?!]", " ", sentence)
    
    return sentence

# Step 3. 데이터 토큰화

In [5]:
from konlpy.tag import Mecab
mecab=Mecab()

def build_corpus(src, tgt, max_len=50):
    """
    소스 문장과 타겟 문장을 입력으로 받아 전처리, 토큰화, 중복 제거를 수행한 후 반환합니다.
    
    Args:
        src (list): 소스 문장 리스트
        tgt (list): 타겟 문장 리스트
        max_len (int): 토큰화된 문장의 최대 길이 (기본값 50)
    
    Returns:
        tuple: 토큰화된 소스 문장 리스트와 타겟 문장 리스트
    """
    # 1. 전처리 및 토큰화
    corpus = []  # 소스-타겟 쌍을 저장할 리스트
    
    for src_sentence, tgt_sentence in zip(src, tgt):
        # 소스 문장 전처리 및 토큰화
        src_tokens = mecab.morphs(preprocess_sentence(src_sentence))
        # 타겟 문장 전처리 및 토큰화
        tgt_tokens = mecab.morphs(preprocess_sentence(tgt_sentence))
        
        # 2. 길이 제한 적용
        if len(src_tokens) <= max_len and len(tgt_tokens) <= max_len:
            src_tokens = ' '.join(src_tokens)  # 공백으로 토큰 결합
            tgt_tokens = ' '.join(tgt_tokens)
            corpus.append((src_tokens, tgt_tokens))  # 소스-타겟 쌍으로 저장
    
    # 3. 중복 제거 (쌍 기준으로 중복 제거)
    corpus = list(set(corpus))
    
    # 4. 소스와 타겟 분리
    src_corpus, tgt_corpus = zip(*corpus)  # 튜플을 두 개의 리스트로 분리
    
    return list(src_corpus), list(tgt_corpus)


In [6]:
# 함수 실행
que_corpus, ans_corpus = build_corpus(questions, answers)

# 결과 확인
print("Tokenized Questions:", que_corpus[:5])
print("Tokenized Answers:", ans_corpus[:5])



Tokenized Questions: ['헤어진지 두 달 무덤덤 한데', '이 발 어떻게 할까', '만나 는 사람 마다 유형 이 비슷 해', '밥 먹 었 는데 도 또 배고파', '친구 따라다녀야 겠다 .']
Tokenized Answers: ['무덤덤 한 데 도 마음 이 허한 가 봅니다 .', '짧 게 변화 를 줘도 괜찮 을 거 같 아요 .', '자신 이 변하 지 않 았 기 때문 이 겠 죠 .', '디 져 트 드세요 .', '친구 가 따라오 게 해 보 세요 .']


In [7]:
print(len(que_corpus))
print(len(ans_corpus))

11747
11747


# Step 4. Augmentation

In [8]:
# pip install gensim==3.8.3

In [9]:
from gensim.models import Word2Vec
model = Word2Vec.load('ko.bin')
wv = model.wv

In [10]:
def lexical_sub(sentence, word_vectors, threshold=0.7, max_attempts=10):
    tokens = sentence.split()
    
    for _ in range(max_attempts):
        selected_tok = random.choice(tokens)
        
        if selected_tok in word_vectors.vocab:  # 'key_to_index' 대신 'vocab' 사용
            similar_words = word_vectors.most_similar(selected_tok, topn=10)
            
            for word, similarity in similar_words:
                if similarity > threshold and word != selected_tok:
                    new_sentence = ' '.join([word if tok == selected_tok else tok for tok in tokens])
                    return new_sentence
    
    return None

def augment_data(corpus, word_vectors):
    augmented_corpus = []
    for sentence in tqdm(corpus):
        new_sentence = lexical_sub(sentence, word_vectors)
        if new_sentence is not None:
            augmented_corpus.append(new_sentence)
        else:
            augmented_corpus.append(sentence)
    return augmented_corpus


In [11]:
# 데이터 증강 수행
augmented_que_corpus = augment_data(que_corpus, wv)
augmented_ans_corpus = augment_data(ans_corpus, wv)

# 최종 데이터셋 구성
final_que_corpus = que_corpus + augmented_que_corpus + que_corpus
final_ans_corpus = ans_corpus + ans_corpus + augmented_ans_corpus

# 최종 데이터셋 구성
final_dataset = []
for i in range(len(que_corpus)):
    # 원본 질문 + 원본 답변
    final_dataset.append((que_corpus[i], ans_corpus[i]))
    
    # 증강된 질문 + 원본 답변
    final_dataset.append((augmented_que_corpus[i], ans_corpus[i]))
    
    # 원본 질문 + 증강된 답변
    final_dataset.append((que_corpus[i], augmented_ans_corpus[i]))


# 데이터 셔플
#random.shuffle(final_dataset)

# 질문과 답변 분리
final_que_corpus, final_ans_corpus = zip(*final_dataset)

100%|██████████| 11747/11747 [02:06<00:00, 92.97it/s] 


In [12]:
print("Original data size:", len(que_corpus))
print("Augmented data size:", len(final_que_corpus))
print("Augmentation ratio:", len(final_que_corpus) / len(que_corpus))

# 결과 확인
print("\nFinal dataset (first 15):")
for i in range(15):
    print(f"Q: {final_que_corpus[i]}")
    print(f"A: {final_ans_corpus[i]}")
    print()

Original data size: 11747
Augmented data size: 35241
Augmentation ratio: 3.0

Final dataset (first 15):
Q: 헤어진지 두 달 무덤덤 한데
A: 무덤덤 한 데 도 마음 이 허한 가 봅니다 .

Q: 헤어진지 두 달 무덤덤 한데
A: 무덤덤 한 데 도 마음 이 허한 가 봅니다 .

Q: 헤어진지 두 달 무덤덤 한데
A: 무덤덤 한 데 도 마음 이 허한 가 봅니다 .

Q: 이 발 어떻게 할까
A: 짧 게 변화 를 줘도 괜찮 을 거 같 아요 .

Q: 이 발 어떻게 할까
A: 짧 게 변화 를 줘도 괜찮 을 거 같 아요 .

Q: 이 발 어떻게 할까
A: 짧 게 변화 를 줘도 괜찮 을 거 같 아요 .

Q: 만나 는 사람 마다 유형 이 비슷 해
A: 자신 이 변하 지 않 았 기 때문 이 겠 죠 .

Q: 만나 는 사람 마다 유형 이 유사 해
A: 자신 이 변하 지 않 았 기 때문 이 겠 죠 .

Q: 만나 는 사람 마다 유형 이 비슷 해
A: 자신 이 바뀌 지 않 았 기 때문 이 겠 죠 .

Q: 밥 먹 었 는데 도 또 배고파
A: 디 져 트 드세요 .

Q: 밥 먹 었 으며 도 또 배고파
A: 디 져 트 드세요 .

Q: 밥 먹 었 는데 도 또 배고파
A: 디 져 트 드세요 .

Q: 친구 따라다녀야 겠다 .
A: 친구 가 따라오 게 해 보 세요 .

Q: 연인 따라다녀야 겠다 .
A: 친구 가 따라오 게 해 보 세요 .

Q: 친구 따라다녀야 겠다 .
A: 친구 가 찾아오 게 해 보 세요 .



# Step 5. 데이터 벡터화

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# 1. 타겟 데이터에 <start>와 <end> 토큰 추가
final_ans_corpus = [['<start>'] + answer + ['<end>'] for answer in final_ans_corpus]

# 전체 코퍼스 생성 (질문과 답변 결합)
total_corpus = list(final_que_corpus) + [' '.join(answer) for answer in final_ans_corpus]

# 2. 단어 사전 구축 및 벡터화
tokenizer = Tokenizer(filters='', oov_token='OOV')
tokenizer.fit_on_texts(total_corpus)

# 질문(인코더 입력) 벡터화
enc_train = tokenizer.texts_to_sequences(final_que_corpus)
enc_train = pad_sequences(enc_train, padding='post')

# 답변(디코더 입력) 벡터화
dec_train = tokenizer.texts_to_sequences([' '.join(answer) for answer in final_ans_corpus])
dec_train = pad_sequences(dec_train, padding='post')

print("단어 사전 크기:", len(tokenizer.word_index))
print("enc_train shape:", enc_train.shape)
print("dec_train shape:", dec_train.shape)


# 단어 사전 저장 (선택사항)
import pickle
with open('word_index.pkl', 'wb') as f:
    pickle.dump(tokenizer.word_index, f)



단어 사전 크기: 7373
enc_train shape: (35241, 32)
dec_train shape: (35241, 46)


# Step 6. 훈련하기

## 트랜스포머 구현

### positional_encoding

In [19]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table
    
print("슝=3")

슝=3


### 마스크 생성

In [20]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask
    
print("슝=3")

슝=3


### MultiHeadAttention

In [21]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights
print("슝=3")

슝=3


### Position-wise Feed Forward Network

In [22]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out
        
print("슝=3")

슝=3


### EncoderLayer

In [23]:
# Encoder의 레이어 구현
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

print("슝=3")

슝=3


### DecoderLayer

In [24]:
# Decoder 레이어 구현
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn
print("슝=3")

슝=3


### Encoder

In [25]:
# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns
print("슝=3")

슝=3


### Decoder

In [26]:
# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns
print("슝=3")

슝=3


### Transformer Layer

In [27]:
class Transformer(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 src_vocab_size,
                 tgt_vocab_size,
                 pos_len,
                 dropout=0.2,
                 shared_fc=True,
                 shared_emb=False):
        super(Transformer, self).__init__()
        
        # 초기화 파라미터 저장
        self.n_layers = n_layers
        self.d_model = tf.cast(d_model, tf.float32)
        self.n_heads = n_heads
        self.d_ff = d_ff
        self.dropout_rate = dropout
        
        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns
print("슝=3")

슝=3


In [28]:
VOCAB_SIZE = len(tokenizer.word_index) + 1  # tokenizer에서 vocab size를 가져옵니다
MAX_LEN = max(enc_train.shape[1], dec_train.shape[1])  # 최대 시퀀스 길이

In [29]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성

transformer = Transformer(
    n_layers=1,
    d_model=368,
    n_heads=8,
    d_ff=1024,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.2,
    shared_fc=True,
    shared_emb=True)
		
d_model = 368

print("슝=3")

슝=3


In [30]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=1000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)
print("슝=3")

슝=3


In [31]:
# Learning Rate 인스턴스 선언 & Optimizer 구현


learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)
print("슝=3")

슝=3


In [32]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
print("슝=3")

슝=3


In [33]:
# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns
print("슝=3")

슝=3


## 훈련하기

In [34]:
# Q. 위의 코드를 활용하여 모델을 훈련시켜봅시다!
EPOCHS = 10
BATCH_SIZE = 64
DATASET_SIZE = len(enc_train)  # 전체 데이터셋 크기를 미리 계산

# 데이터셋 생성
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
train_dataset = train_dataset.shuffle(DATASET_SIZE).batch(BATCH_SIZE)

STEPS_PER_EPOCH = DATASET_SIZE // BATCH_SIZE  # 에폭당 스텝 수 계산

for epoch in range(EPOCHS):
    total_loss = 0
    
    tqdm_bar = tqdm(total=STEPS_PER_EPOCH)  # 스텝 수를 tqdm에 전달
    
    for (batch, (src, tgt)) in enumerate(train_dataset):
        batch_loss, _, _, _ = train_step(src, tgt, transformer, optimizer)
        total_loss += batch_loss
        tqdm_bar.update(1)
        tqdm_bar.set_postfix({
            'Epoch': f'{epoch+1}/{EPOCHS}',
            'Loss': f'{batch_loss.numpy():.4f}'
        })
    
    tqdm_bar.close()  # tqdm 객체 닫기
    
    epoch_loss = total_loss / STEPS_PER_EPOCH
    print(f'\nEpoch {epoch+1} Loss: {epoch_loss:.4f}\n')


551it [00:28, 19.62it/s, Epoch=1/10, Loss=1.6048]                         



Epoch 1 Loss: 2.8268



551it [00:22, 24.53it/s, Epoch=2/10, Loss=1.0318]                         



Epoch 2 Loss: 1.1765



551it [00:22, 24.41it/s, Epoch=3/10, Loss=0.4144]                         


...


Epoch 8 Loss: 0.1512



551it [00:24, 22.94it/s, Epoch=9/10, Loss=0.1008]                         



Epoch 9 Loss: 0.1327



551it [00:24, 22.70it/s, Epoch=10/10, Loss=0.1120]                         


Epoch 10 Loss: 0.1212



## 챗봇 대답 생성

In [35]:
def translate(sentence, model, tokenizer):
    tokens = tokenizer.texts_to_sequences([sentence])[0]
    padded_tokens = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=MAX_LEN,
                                                           padding='post')
    output = tf.expand_dims([tokenizer.word_index['<start>']], 0)   
    for i in range(MAX_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(padded_tokens, output)

        predictions, _, _, _ = model(padded_tokens, 
                                     output,
                                     enc_padding_mask,
                                     combined_mask,
                                     dec_padding_mask)

        predicted_id = tf.argmax(predictions[:, -1:, :], axis=-1).numpy()[0][0]

        if tokenizer.index_word[predicted_id] == '<end>':
            break

        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    # <start> 토큰을 제외하고 결과 생성
    result = ' '.join([tokenizer.index_word[i] for i in output.numpy()[0][1:] if i not in [0, tokenizer.word_index['<start>']]])
    
    # 마지막에 <end> 토큰 추가
    result = result + ' <end>'
    
    return result


print("슝=3")

슝=3


In [36]:
# 예문 번역 및 결과 출력
examples = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

print("# 제출\n")
print("Translations")
for i, example in enumerate(examples, 1):
    translation = translate(example, transformer, tokenizer)
    print(f"> {i}. {translation}")



print("\nHyperparameters")
print(f"> n_layers: {transformer.n_layers}")
print(f"> d_model: {transformer.d_model.numpy()}")
print(f"> n_heads: {transformer.n_heads}")
print(f"> d_ff: {transformer.d_ff}")
print(f"> dropout: {transformer.dropout_rate}")

print("\nTraining Parameters")
print(f"> Warmup Steps: {learning_rate.warmup_steps}")
print(f"> Batch Size: {BATCH_SIZE}")
print(f"> Epoch At: {EPOCHS}")


# 제출

Translations
> 1. 이제 보 지 마세요 . <end>
> 2. 오늘 도 고생 이 많 았 어요 . <end>
> 3. 오래 살 면 정말 떨리 죠 . <end>
> 4. 이제 보 지 마세요 . <end>

Hyperparameters
> n_layers: 1
> d_model: 368.0
> n_heads: 8
> d_ff: 1024
> dropout: 0.2

Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10


# Step 7. 성능 측정하기

In [37]:
import pandas as pd
import random
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm


def calculate_bleu(reference, candidate):
    # Mecab으로 토큰화
    ref_tokens = mecab.morphs(reference)
    can_tokens = mecab.morphs(candidate)
    
    return sentence_bleu([ref_tokens],
                         can_tokens,
                         smoothing_function=SmoothingFunction().method1)


def eval_bleu(model, samples, tokenizer):
    total_score = 0.0
    sample_size = len(samples)
    
    print("Translations and BLEU Scores")
    for idx, (question, answer) in enumerate(samples, 1):
        translation = translate(question, model, tokenizer)
        
        # 원문 및 결과 토큰화 출력
        ref_tokens = mecab.morphs(answer)
        can_tokens = mecab.morphs(translation)
        
        bleu_score = calculate_bleu(answer, translation)
        total_score += bleu_score
        
        print(f"\n> {idx}. Question: {question}")
        print(f"   Original Answer: {answer}")
        print(f"   Original Tokens: {ref_tokens}")  # 토큰화 결과 출력
        print(f"   Model Translation: {translation}")
        print(f"   Model Tokens: {can_tokens}")  # 토큰화 결과 출력
        print(f"   BLEU Score: {bleu_score:.4f}")
    
    avg_bleu_score = total_score / sample_size
    print(f"\nNumber of Samples: {sample_size}")
    print(f"Average BLEU Score: {avg_bleu_score:.4f}")
    
    return avg_bleu_score

In [38]:
# 무작위로 샘플 선택
num_samples = 10  # 선택할 샘플 수
samples = df.sample(n=num_samples)[['Q', 'A']].values.tolist()

# BLEU 점수 계산
print("\nBLEU Score Evaluation")
avg_bleu = eval_bleu(transformer, samples, tokenizer)

print(f"\nFinal Average BLEU Score: {avg_bleu:.4f}")

print("\nHyperparameters")
print(f"> n_layers: {transformer.n_layers}")
print(f"> d_model: {transformer.d_model.numpy()}")
print(f"> n_heads: {transformer.n_heads}")
print(f"> d_ff: {transformer.d_ff}")
print(f"> dropout: {transformer.dropout_rate}")

print("\nTraining Parameters")
print(f"> Warmup Steps: {learning_rate.warmup_steps}")
print(f"> Batch Size: {BATCH_SIZE}")
print(f"> Epoch At: {EPOCHS}")


BLEU Score Evaluation
Translations and BLEU Scores

> 1. Question: 좋아하는 여자가 다른 남자랑 친해지는게 짜증남.
   Original Answer: 충분히 신경쓰인 부분이에요.
   Original Tokens: ['충분히', '신경', '쓰인', '부분', '이', '에요', '.']
   Model Translation: 때 의 마음 이 깊 네요 . <end>
   Model Tokens: ['때', '의', '마음', '이', '깊', '네요', '.', '<', 'end', '>']
   BLEU Score: 0.0251

> 2. Question: 원래 다 이런거겠죠
   Original Answer: 누구나 그럴 거예요.
   Original Tokens: ['누구', '나', '그럴', '거', '예요', '.']
   Model Translation: 누구 나 한테 도 잊 으세요 . <end>
   Model Tokens: ['누구', '나', '한테', '도', '잊', '으세요', '.', '<', 'end', '>']
   BLEU Score: 0.0494

> 3. Question: 맨날 똑같애
   Original Answer: 똑같은 건 없어요.
   Original Tokens: ['똑같', '은', '건', '없', '어요', '.']
   Model Translation: 때론 묻어두 는 게 좋 을 때 도 있 어요 . <end>
   Model Tokens: ['때론', '묻어두', '는', '게', '좋', '을', '때', '도', '있', '어요', '.', '<', 'end', '>']
   BLEU Score: 0.0302

> 4. Question: 입소문이 무서워
   Original Answer: 긍정적이고 부정적인 파급력이 모두 세죠.
   Original Tokens: ['긍정', '적', '이', '고', '부정', '적', '인', '파급', '력', 

# 회고

4. Question: 입소문이 무서워
   Original Answer: 긍정적이고 부정적인 파급력이 모두 세죠.
   Original Tokens: ['긍정', '적', '이', '고', '부정', '적', '인', '파급', '력', '이', '모두', '세', '죠', '.']
   Model Translation: 긍정 적 이 고 부정 적 인 파급 력 이 모두 세 죠 . <end>
   Model Tokens: ['긍정', '적', '이', '고', '부정', '적', '인', '파급', '력', '이', '모두', '세', '죠', '.', '<', 'end', '>']
   BLEU Score: 0.8053

<end> 토큰 처리 후 다시 점수를 계산해야할듯,,!